In [2]:
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
import requests # Импортируем библиотеку requests


Для поиска необходимых нам данных мы будем использовать библиотеку BeautifulSoup, которая позволяет по названию тегов и их атрибутов получать содержащийся в них текст.

BeautifulSoup не является частью стандартной библиотеки, поэтому для начала её нужно установить. Например, в Jupyter Notebook это делается с помощью такой команды:

# Устанавливаем библиотеку BeautifulSoup
!pip install beautifulsoup4 
После установки импортируем библиотеку в наш код:

from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup

Теперь мы можем извлекать данные из любой веб-страницы.

Ранее мы уже получили содержимое страницы с помощью GET-запроса и сохранили информацию в переменной response , теперь создадим объект BeautifulSoup с именем page, указывая в качестве параметра html.parser.

Для примера получим информацию o title (с англ. заголовок) — это строка, которая отображается на вкладке браузера:

In [3]:
url = 'https://nplus1.ru/news/2021/10/11/econobel2021' # Определяем адрес страницы
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
print(page.title) # Получаем тег title, отображающийся на вкладке браузера
print(page.title.text) # Выводим текст из полученного тега, который содержится в атрибуте text

<title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>
Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей


Если при запросе к сайту, а затем при его разборе с помощью BeautifulSoup в тексте страницы не находится нужный тег, попробуйте вывести на печать пару тысяч символов текста страницы. Если там обнаружится нечто похожее на капчу, возможно, сайт посчитал вас роботом и отказывается выдавать содержимое. Чтобы получить его, попробуйте «притвориться» браузером при запросе из скрипта:

requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
User-Agent своего браузера можно узнать по этой ссылке.

ИЗВЛЕКАЕМ ЗАГОЛОВОК СТАТЬИ

Выполним поставленную ранее задачу: получить информацию о странице и извлечь заголовок статьи, опубликованной на этой странице, дату публикации, а также текст статьи.

Предположим, что мы знаем, что в HTML-коде рассматриваемой нами страницы заголовок статьи заключён в тег <h1> … </h1> (заголовок первого уровня).

Тогда мы можем получить его текст с помощью метода find() (с англ. найти) объекта BeautifulSoup, передав ему название интересующего нас тега:

In [4]:
# Применяем метод find() к объекту и выводим результат на экран
print(page.find('h1').text)


            Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей
          


о как же узнать, в каких именно тегах заключена необходимая информация?

Проще всего это сделать с помощью так называемого инструмента разработчика, который есть во всех современных браузерах. Покажем, как открыть данный инструмент на примере использования браузера Google Chrome.

Устанавливаем курсор на элементе страницы (заголовок статьи), информацию о котором хотим получить, нажимаем на правую клавишу мыши и в выпадающем списке выбираем пункт «Просмотреть код элемента» или «Исследовать» в зависимости от браузера.



НЕУНИКАЛЬНЫЕ ТЕГИ: ИЗВЛЕКАЕМ ТЕКСТ И ДАТУ ПУБЛИКАЦИИ СТАТЬИ

Теперь получим сам текст статьи. Как вы уже знаете, первым делом необходимо определить, в какой тег он заключён. Применим, как и ранее, инструмент разработчика.



Видим, что искомый текст заключён в тег  <div> … </div> . Попробуем извлечь его уже известным нам способом — с помощью метода find() — и выведем его на экран.

print(page.find('div').text) # Выводим содержимое атрибута text тега div
Мы увидели не то, что ожидали — кучу текста, не имеющего отношения к тому, что мы искали...

?
В чём же проблема?

Дело в том, что теги <div> … </div> очень распространённые и на странице их очень много. Метод find() нашёл первый из них, но это не то, что нам надо.

Посмотрим на нашу страницу, используя инструмент разработчика, ещё раз. Можем заметить, что у искомого текста есть свой класс — n1_material text-18 :

Передадим название класса в метод find() с помощью аргумента class_ и получим текст статьи:

print(page.find('div', class_='n1_material text-18').text) # Выводим содержимое атрибута text тега div класса n1_material text-18


В данном случае происходит поиск точного строкового значения class атрибута, т. е. выполнение строк кода:

print(page.find('div', class_='n1_material').text)
print(page.find('div', class_='n1_material text-18').text)
даст одинаковый результат.

При выполнении строки кода

print(page.find('div', class_='text-18 n1_material').text)
мы получим ошибку, так как такого строкового значения в области поиска нет.

Аналогично получим информации о теге, который содержит дату написания статьи, отображаемую в левом верхнем углу страницы.



Итак, нам нужен тег <a> … </a> с классом "relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center". Для поиска достаточно указать в качестве класса "relative", отбросив дополнительные настройки.

Теперь получим данные из него с помощью уже известного метода find(), передав название нужного тега:

# Выводим на экран содержимое атрибута text тега a с классом "relative"
print(page.find('a', class_= "relative").text)


О поиске по классу можно узнать подробнее в Beautiful Soup Documentation.

Задача решена — мы извлекли из контента страницы заголовок статьи, опубликованной на странице, дату публикации, а также текст статьи.

СБОР НЕСКОЛЬКИХ ЭЛЕМЕНТОВ: СОБИРАЕМ ВСЕ ССЫЛКИ НА СТРАНИЦЕ

Рассмотрим ещё один сценарий: вы хотите собрать сразу несколько элементов со страницы. Например, представьте, что вы хотите получить названия всех языков программирования, упомянутых на странице в Wikipedia в статье про языки программирования.

Можно заметить, что все названия языков программирования на этой странице связаны ссылками c соответствующими статьями о них. Таким образом, нам необходимо собрать все ссылки на странице. Для ссылок в HTML предусмотрен тег <a> … </a>. Попробуем использовать find():

url = 'https://en.wikipedia.org/wiki/List_of_programming_languages' # Задаём адрес ресурса
response = requests.get(url) # Делаем GET-запрос к ресурсу
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup
print(page.find('a')) # Ищем ссылку по тегу <a> и выводим её на экран


Мы получили только одну ссылку, хотя на странице их явно больше.

Это происходит, потому что метод find() возвращает только первый подходящий элемент. Если требуется получить больше элементов, необходимо воспользоваться методом find_all() (с англ. найти все):

links = page.find_all('a') # Ищем все ссылки на странице и сохраняем в переменной links в виде списка
print(len(links)) # Выводим количество найденных ссылок

Итак, на момент создания этих учебных материалов на странице содержалось 928 ссылок. Посмотрим на некоторые из них:

print([link.text for link in links[500:510]]) # Выводим ссылки с 500 по 509 включительно


Не все ссылки соответствуют названиям языков программирования — страница содержит также «служебные» ссылки, такие, например, как Jump to navigation (с англ. Перейти к навигации) или Alphabetical (с англ. По алфавиту):

print([link.text for link in links[0:10]]) # Выводим ссылки с 1 по 9 включительно


Для обработки полученных данных и исключения «лишней» информации можно, например, использовать подходы, которые вы изучили в модуле PYTHON-14.

✍ В заключение заметим, что BeautifulSoup — достаточно мощная библиотека. Мы рассмотрели её базовые возможности, но их полный список гораздо шире. С ним можно ознакомиться в официальной документации.